In [ ]:
'''
SELECT * 
FROM shark_attacks as s
WHERE st_intersects(s.geometry, 
					 st_transform(
						 st_buffer(
							 st_transform(
								 st_setsrid(
									 st_makepoint(-156.767903, 20.749255),4326),26904),50),4326)
				   )

SELECT * 
FROM bottom_type as b
WHERE st_intersects(b.geometry, 
					 st_transform(
						 st_buffer(
							 st_transform(
								 st_setsrid(
									 st_makepoint(-156.076333, 20.815275),4326),26904),10000),4326)
				   )'''